In [1]:
#Importando todas as bibliotecas necessárias
import pandas as pd
import numpy as np 
from pandas.plotting import scatter_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import metrics


import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
 

In [2]:
'''#Carregando o arquivo com a biblioteca do pandas que está guardado na variável pd
dataSetLOL = 'E://ESTUDANDO//TCC//_CAPITULO_4_LOL//LOL//NOVO_LOL//games.csv'
lol = pd.read_csv(dataSetLOL, sep =',')

#Função necessária para mostrar todas as linhas e colunas do dataset sem truncá-las
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#Exibe as 5 primeiras linhas do dataset para fins de verificação
lol.head()'''

"#Carregando o arquivo com a biblioteca do pandas que está guardado na variável pd\ndataSetLOL = 'E://ESTUDANDO//TCC//_CAPITULO_4_LOL//LOL//NOVO_LOL//games.csv'\nlol = pd.read_csv(dataSetLOL, sep =',')\n\n#Função necessária para mostrar todas as linhas e colunas do dataset sem truncá-las\npd.set_option('display.max_columns', None)\npd.set_option('display.max_rows', None)\n\n#Exibe as 5 primeiras linhas do dataset para fins de verificação\nlol.head()"

In [3]:
#Carregando o arquivo com a biblioteca do pandas que está guardado na variável pd
dataSetLOL = 'LeagueOfLedends2017.xlsx'
lol = pd.read_excel(dataSetLOL, encoding='latin1')

#Função necessária para mostrar todas as linhas e colunas do dataset sem truncá-las
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



In [4]:
#Segundo a função shape, temos 37404 linhas e 98 colunas no dataset
lol.shape

(37404, 98)

<h1>Chances de vitória: Variáveis de interesse</h1>
<p><h3>Queremos checar quais times venceram a partida - "result" = 1 na tabela -, usando como parâmetro, variáveis de interesse.</h3></p>
<p>As variáveis de interesse para a análise das chances de vitória para esse caso de uso são:</p> 
<ul>
    <li><b>gameID:        </b> Identificador da partida, único para cada servidor/liga.</li>
    <li><b>side:          </b> Lado em que o time analisado começou na partida (Azul ou Vermelho).</li>
    <li><b>champion:      </b> Campeões escolhidos na partida pelo time.</li>
    <li><b>position:      </b> Posição em que os campeões jogam na partida.</li>
    <li><b>team:          </b> Nome do time na partida.</li>
    <li><b>ft:            </b> Informa se o time foi o primeiro a derrubar uma torre inimiga.</li>
</ul>
    <p>Entretando, para utilizar as variáveis para a criação de um modelo preditivo, é necessário primeiramente, trabalhá-las adequadamente.</p>

<h1>Funcionamento das partidas:</h1>
<p>Devido a natureza de como se dá o funcionamento das partidas de League of Legends, com 5 jogadores para cada um dos dois times, cada jogar escolhendo seu campeão e sua posição na partida, colunas novas serão criadas em um novo dataframe identificando qual campeão cada jogador dos dois times pegou na partida e qual posição cada um pegou.</p>

<h3>Criando funções de padronização</h3>
<p>Abaixo serão criadas funções cujo objetivo será padronizar as colunas "team" e "champion" que provavelmente possui muitos valores inválidos</p>

In [5]:
'''
Aqui todos os campeões estão sendo adicionados em uma lista de palavras e as mesmas estão sendo limpas:
todos os caracteres minúsculos, eliminando todas os espaços em branco, eliminando todas as pontuações,
removendo todas as palavras duplicadas ( com o dict). No final, todas as palavras modificadas serão adicionados em 
uma lista de campeões. Essa limpeza será uma nova funcionalidade criada em uma função chamada "padronizar_campeoes"
'''
import string

def padronizar_campeoes(champions):
    champions = champions.lower().translate(str.maketrans('', '', string.punctuation)).replace(' ', '').replace('-', '').strip('')
    return champions

In [6]:
'''
Aqui todos os times estão sendo adicionados em uma lista de palavras e as mesmas estão sendo limpas:
todos os caracteres minúsculos, eliminando todas os espaços em branco, eliminando todas as pontuações,
removendo todas as palavras duplicadas ( com o dict). No final, todas as palavras modificadas serão adicionados em 
uma lista de times. Essa limpeza será uma nova funcionalidade criada em uma função chamada "padronizar_times"
'''
import string

def padronizar_times(teams):
    teams = teams.lower().translate(str.maketrans('', '', string.punctuation)).replace(' ', '').replace('-', '').strip('')
    return teams

<h1>Aplicando as funções linha a linha:</h1>
<p>O objetivo agora é aplicar essas funções linha a linha, passando a variável que queremos alterar como parâmetro. Para fazer isso,
usarei o método "map", passando como argumento o nome da função e atribuindo isso a uma nova coluna que será criada com os 
dados alterados. As novas colunas criadas terão o mesmo nome da coluna original, mas o o 's' na frente de standard</p>

In [7]:
'''
Aplicando a função de padronização na coluna "champion". Uma nova coluna será criada e 
já padronizada, chamada "s_champion"
'''
lol["s_champion"] = lol["champion"].map(padronizar_campeoes)


In [8]:
'''
Aplicando a função de padronização na coluna "team". Uma nova coluna será criada e 
já padronizada, chamada "s_team"
'''
lol["s_team"] = lol["team"].map(padronizar_times)


<h3>Criando um novo dataframe</h3>
<p>Para facilitar na utilização dos modelos preditivos de Machine Learning e na economia de memória, não precisamos das quase 100 colunas do dataset. Portanto, um novo dataframe será criado contendo apenas as colunas necessárias.</p>

In [9]:
'''
Criando uma nova variável contendo somente as variáveis de interesse
'''
w_lol = pd.concat([ lol["s_team"], lol["gameid"], lol["side"], lol["result"],
                   lol['ft'],lol['fbaron'], lol["s_champion"], lol["position"],
                   lol["csdat10"], lol["gdat10"], lol["xpdat10"], lol["gdat15"] ], axis = 1).copy()

'''w_lol = pd.concat([ lol["s_team"], lol["gameid"], lol["side"], lol["result"],
                   lol['ft'],lol['fbaron'], lol["s_champion"], lol["position"],
                   lol["csdat10"], lol["gdat10"], lol["xpdat10"], lol["gdat15"] ], axis = 1).copy()
'''
'''w_lol = pd.concat([ lol["s_team"], lol["gameid"], lol["side"], lol["result"],
                   lol["s_champion"], lol["position"],
                   lol["csdat10"], lol["gdat10"], lol["xpdat10"], lol["gdat15"] ], axis = 1).copy()'''


'w_lol = pd.concat([ lol["s_team"], lol["gameid"], lol["side"], lol["result"],\n                   lol["s_champion"], lol["position"],\n                   lol["csdat10"], lol["gdat10"], lol["xpdat10"], lol["gdat15"] ], axis = 1).copy()'

In [10]:
w_lol.head(10)

,s_team,gameid,side,result,ft,fbaron,s_champion,position,csdat10,gdat10,xpdat10,gdat15
0,teamliquid,1002300127,Blue,0,1.0,0.0,gnar,Top,5.0,114.0,-8.0,123.0
1,teamliquid,1002300127,Blue,0,1.0,0.0,gragas,Jungle,0.0,53.0,-27.0,-756.0
2,teamliquid,1002300127,Blue,0,1.0,0.0,ekko,Middle,1.0,-22.0,-81.0,-1050.0
3,teamliquid,1002300127,Blue,0,1.0,0.0,vayne,ADC,-7.0,-354.0,157.0,-1057.0
4,teamliquid,1002300127,Blue,0,1.0,0.0,lulu,Support,3.0,-133.0,-223.0,-1114.0
5,dignitas,1002300127,Red,1,0.0,1.0,jarvaniv,Top,-5.0,-114.0,8.0,-123.0
6,dignitas,1002300127,Red,1,0.0,1.0,maokai,Jungle,0.0,-53.0,27.0,756.0
7,dignitas,1002300127,Red,1,0.0,1.0,taliyah,Middle,-1.0,22.0,81.0,1050.0
8,dignitas,1002300127,Red,1,0.0,1.0,varus,ADC,7.0,354.0,-157.0,1057.0
9,dignitas,1002300127,Red,1,0.0,1.0,janna,Support,-3.0,133.0,223.0,1114.0


<h3>Padronizando valores em strings </h3>
<p> Os modelos preditivos aceitam apenas valores numéricos. Portanto, as variáveis de interesse cujos valores são strings precisam ser padronizadas. Existem 3 variáveis em que esses problemas ocorrem: 'champion','side', 'position'. Para realizar esse processo de padronização, uma função foi criada com o objetivo de utilizar o método "LabelEncoder" pertencente a biblioteca sklearn.preprocessing. Esse método foi explicitado no site "StackOverflow" por um usuário, no seguinte link: https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn <p>

In [11]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

class PadronizacaoStrings:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self 

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [12]:
'''
Aqui, a função de padronização de Strings será chamado e esse processo será atribuído a um novo
dataframe cujo nome é "s_lol"
'''
s_lol = (PadronizacaoStrings(columns = ['side', 's_champion', 's_team', 'position']).fit_transform(w_lol)).copy()


In [13]:
s_lol.head()

,s_team,gameid,side,result,ft,fbaron,s_champion,position,csdat10,gdat10,xpdat10,gdat15
0,96,1002300127,0,0,1.0,0.0,32,5,5.0,114.0,-8.0,123.0
1,96,1002300127,0,0,1.0,0.0,33,1,0.0,53.0,-27.0,-756.0
2,96,1002300127,0,0,1.0,0.0,23,2,1.0,-22.0,-81.0,-1050.0
3,96,1002300127,0,0,1.0,0.0,113,0,-7.0,-354.0,157.0,-1057.0
4,96,1002300127,0,0,1.0,0.0,63,3,3.0,-133.0,-223.0,-1114.0


<h3>Mudando novamente o dataframe para a forma final</h3>
<p>Aqui acontece a mudança mais essencial ao dataframe. Atualmente ele está estruturado da seguinte forma: Cada jogo possui o seu id único (gameid). Para cada jogo, temos listados 5 jogadores por linha, para cada um dos dois times. Logo depois, temos mais duas linhas listando os dois times participantes da partida. Ou seja, para cada partida, temos um total de 12 linhas representando cada jogador, seus times e suas posições na partida. A mudança feita no dataframe foi alocar as informações contidas nessas 12 linhas para uma única linha, aumentando o número de colunas de 6 para 24</p>

In [14]:
'''
Aqui acontece a transformação mais importante do dataframe. 
'''
i = 0

result = {}

for index, value in s_lol.iterrows():
    if (not(value.gameid in result)):
        result[value.gameid] = {
            "result": value.result,
            "side": value.side,
            "team": value.s_team,
            "gameid": value.gameid,
            "ft" : value.ft,
            "fbaron" : value.fbaron
        }
        i = 0
    if i == 10:
        continue
    
    if (i < 5):
        result[value.gameid]["champion"+str(i+1)] = value.s_champion
        result[value.gameid]["position"+str(i+1)] = value.position
        result[value.gameid]["csdif"+str(i+1)] = value.csdat10
        result[value.gameid]["golddif10_"+str(i+1)] = value.gdat10
        result[value.gameid]["xpdif"+str(i+1)] = value.xpdat10
        result[value.gameid]["golddif15_"+str(i+1)] = value.gdat15
    else:
        result[value.gameid]["enemy_champion"+str(i-4)] = value.s_champion
        result[value.gameid]["enemy_position"+str(i-4)] = value.position

    
    i = i + 1

    
df_lol = pd.DataFrame.from_dict(result, orient='index')


    


In [15]:
s_lol.head(10)

,s_team,gameid,side,result,ft,fbaron,s_champion,position,csdat10,gdat10,xpdat10,gdat15
0,96,1002300127,0,0,1.0,0.0,32,5,5.0,114.0,-8.0,123.0
1,96,1002300127,0,0,1.0,0.0,33,1,0.0,53.0,-27.0,-756.0
2,96,1002300127,0,0,1.0,0.0,23,2,1.0,-22.0,-81.0,-1050.0
3,96,1002300127,0,0,1.0,0.0,113,0,-7.0,-354.0,157.0,-1057.0
4,96,1002300127,0,0,1.0,0.0,63,3,3.0,-133.0,-223.0,-1114.0
5,17,1002300127,1,1,0.0,1.0,41,5,-5.0,-114.0,8.0,-123.0
6,17,1002300127,1,1,0.0,1.0,67,1,0.0,-53.0,27.0,756.0
7,17,1002300127,1,1,0.0,1.0,102,2,-1.0,22.0,81.0,1050.0
8,17,1002300127,1,1,0.0,1.0,112,0,7.0,354.0,-157.0,1057.0
9,17,1002300127,1,1,0.0,1.0,40,3,-3.0,133.0,223.0,1114.0


<h3>Mudança de dimensionalidade:</h3>
    <p>Como era esperado, a dimensionalidade mudou drasticamente. Isso ocorre porque todas as linhas foram divididas em vários pedaços diferentes. E esses pedaços foram todos alocados em 20 novas colunas que representa cada posição e cada campeão escolhido por cada um dos 10 jogadores ( dos dois times) da partida. </p>

In [16]:
w_lol.shape

(37404, 12)

In [17]:
df_lol.shape

(3090, 46)

<h3>Verificando a existência de dados nulos</h3>

In [18]:
#Verificando a quantidade de dados nulos em cada coluna
df_lol.isnull().sum()

result               0
side                 0
team                 0
gameid               0
ft                 534
fbaron             612
champion1            0
position1            0
csdif1             534
golddif10_1        534
xpdif1             534
golddif15_1        534
champion2            0
position2            0
csdif2             534
golddif10_2        534
xpdif2             534
golddif15_2        534
champion3            0
position3            0
csdif3             534
golddif10_3        534
xpdif3             534
golddif15_3        534
champion4            0
position4            0
csdif4             534
golddif10_4        534
xpdif4             534
golddif15_4        534
champion5            0
position5            0
csdif5             534
golddif10_5        534
xpdif5             534
golddif15_5        534
enemy_champion1      0
enemy_position1      0
enemy_champion2      0
enemy_position2      0
enemy_champion3      0
enemy_position3      0
enemy_champion4      0
enemy_posit

In [19]:
'''
Como verificado acima, temos alguns valores nulos no dataset, referentes a coluna 'ft'. Os valores
nulos representam um total de 17,28% do dataframe. Segundo o site 
(https://analyticsindiamag.com/5-ways-handle-missing-values-machine-learning-datasets/), se os 
valores nulos representam até 30% no máximo o total de informação do dataset, é seguro apenas
excluir os valores nulos. Portanto, os valores nulos terão suas linhas deletadas

'''
df_lol.dropna(inplace = True)
df_lol.isnull().sum()

result             0
side               0
team               0
gameid             0
ft                 0
fbaron             0
champion1          0
position1          0
csdif1             0
golddif10_1        0
xpdif1             0
golddif15_1        0
champion2          0
position2          0
csdif2             0
golddif10_2        0
xpdif2             0
golddif15_2        0
champion3          0
position3          0
csdif3             0
golddif10_3        0
xpdif3             0
golddif15_3        0
champion4          0
position4          0
csdif4             0
golddif10_4        0
xpdif4             0
golddif15_4        0
champion5          0
position5          0
csdif5             0
golddif10_5        0
xpdif5             0
golddif15_5        0
enemy_champion1    0
enemy_position1    0
enemy_champion2    0
enemy_position2    0
enemy_champion3    0
enemy_position3    0
enemy_champion4    0
enemy_position4    0
enemy_champion5    0
enemy_position5    0
dtype: int64

In [20]:
df_lol.shape

(2478, 46)

<h3>Dividindo o Dataframe: </h3>
<p>A divisão feita pelo train_test_split é aleatória mas é feito somente uma única vez. Para garantir a aleatoriedade do processo e a separação igualitária do dataframe em treino e teste, podemos dividir o dataframe em pecaços iguais ("folds"), embaralhar cada pedaço ("shuffle") e aplicar a divisão em cada pedaço. Para fazer isso, precisamos usar o método "Kfold" pertencente a biblioteca "sklearn.model_selection"</p>

<h3>Calculando a acurácia:</h3>
<p>Após as variáveis de interesse serem treinadas, elas serão usadas para tentar prever a vitória da partida comparando com as variáveis de teste. A acurácia, medirá a taxa de acerto das variáveis de interesse na previsão sobre as chances de vitória da partida. O cálculo da acurácia será feito por cada divisão feita pelo método kfold. No final, será calculado a <b>média da acurácia de todas as  divisões do dataframe feita pelo método kfold</b>. Essa será a acurácia final, tendo somente a variável de interesse calculada no momento como parâmetro determinístico para a vitória da partida. </p>

In [21]:
from sklearn.model_selection import KFold


<h3>Testando múltiplos modelos preditivos em função de sua performance: </h3>
<p>Antes de realizar os processos de cálculo das chances de vitória em função das variáveis de interesse através dos modelos preditivos, primeiramente é necessário definir qual é o modelo preditivo que possui melhor perfomance para a construção de análises preditivas. 4 modelos serão analisados com todas as variáveis possíveis. Entretanto, no final, somente um modelo será usado para todas as variáveis. São eles: KNEIGHBORS_CLASSIFIER, RANDOM_FORES_CLASSIFIER, DECISION_TREE_CLASSIFIER, NAIVE_BAYES. </p>

<h3>Cross Validation Score NÃO treina (fit) o dataset para futuras previsões</h3>
<p>
    De acordo com uma dúvida publicada no Stack Overflow (https://stackoverflow.com/questions/42263915/using-sklearn-cross-val-score-and-kfolds-to-fit-and-help-predict-model?rq=1) o método cross_validation_score serve apenas estimar quais modelos predivitos possuem a melhor performance. Isso é feito a partir da subdivisão do dataset de treino ( o dataset de teste original não é trabalhado), em vários folds ( pedaços) onde 1 dos pedaços será usado como teste. Isso será feito k vezes ( k é passado como parâmetro), sendo que em cada iteração, 1 novo pedaço do dataset de treino será escolhido para testar a acurácia do mesmo. No final, a média da acurácia das k iterações será feita e o resultado da acurácia será estimado.  O dataset de teste original não é usado é porque o objetivo é generalizar a previsão para análises futuras. Não queremos que o algoritmo decore os resultados do dataset de teste. .</p>

In [ ]:
df_lol.head()

In [ ]:
'''
A variável X recebe todas as linhas do dataframe e todas as colunas ( menos a coluna alvo "result").
A variável y recebe somente a coluna "result"
'''

X = np.array(df_lol.iloc[:,4:])
y = np.array(df_lol['result'])


In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn import model_selection
import warnings
warnings.filterwarnings('ignore')

'''
Aqui os modelos preditivos estão sendo testados em função de sua performance EM TODAS AS VARIÁVES
DE INTERESSE. No final, somente 1 modelo preditivo será escolhido para ser usado. O método utilizado
é dividir o dataset em 10 pedaços e calcular a acurácia de cada pedaço. No final calcularemos média 
da acurácia desses 10 pedaços. Quanto maior a média final, melhor o modelo preditivo. Importante 
salientar que o treinamento não está sendo feito aqui. Apenas estimativas de quais modelos funcionam
melhor. 

'''

models = []
models.append(('RANDOM_FOREST_CLASSIFIER', RandomForestClassifier(n_estimators=500, random_state = 0)))
#models.append(('LOGISTIC_REGRESSION', LogisticRegression(solver='liblinear', multi_class='ovr')))
#models.append(('LINEAR_DISCRIMINATION_ANALISYS', LinearDiscriminantAnalysis()))
models.append(('KNEIGHBORS_CLASSIFIER', KNeighborsClassifier()))
models.append(('DECISION_TREE_CLASSIFIER', DecisionTreeClassifier()))
models.append(('NAIVE_BAYS', GaussianNB()))
#models.append(('RIDGE', Ridge()))
#models.append(('STANDARD_SCALLER', StandardScaler()))
#models.append(('DECISION_TREE_REGRESSOR',DecisionTreeRegressor(max_depth=8, min_samples_leaf=0.13, random_state=3)))
#models.append(('SVM', SVC(gamma='auto')))



# evaluate each model in turn
results = []
names = []
plota_names = []
plota_acuracia = []



for name, model in models:
    kf = KFold(10, shuffle=True, random_state = 1)
    hit_rate = 0
    for training_lines, testing_lines in kf.split(X,y):
       
        kf = model_selection.KFold(10, shuffle=True, random_state = 1)
        X_train, X_test = X[training_lines], X[testing_lines]
        y_train, y_test = y[training_lines], y[testing_lines]

        cv_results = cross_val_score(model, X_train, y_train, cv=kf, scoring='accuracy')
        results.append(cv_results)
        names.append(name)

        print('%s => Accuracy: %f, Standard Deviation: (%f)' % (name, cv_results.mean(), cv_results.std()))

        hit_rate += cv_results.mean()
        
    #plota_names.append(names)
    #plota_acuracia.append(hit_rate/10)

    final_accuracy = hit_rate/10   
    print("Média das 10 acurácias: ",final_accuracy)
    #print("Desvio padrão da média final :",hit_rate.std())
    print()



<h3>Variável de interesse com maior influência nas chances de vitória</h3>
<p>Qual variável de interesse exerce maior influência no cálculo das vitórias? Quais os relacionamentos entre as variáveis de interesse? Essas respostas serão dada abaixo. </p>

In [ ]:
fields_to_correlate = ['side', 'team', 'champion1', 'position1', 'champion2', 'position2', 'champion3', 'position3', 'champion4', 'position4', 'champion5', 'position5', 'enemy_champion1', 'enemy_position1', 'enemy_champion2', 'enemy_position2', 'enemy_champion3', 'enemy_position3', 'enemy_champion4', 'enemy_position4', 'enemy_champion5', 'enemy_position5']#.split()


In [ ]:
'''
https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b

As seen from below code, the optimum number of features is 10. We now feed 10 
as number of features to RFE and get the final set of features given by RFE method, as follows:

'''


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso


#no of features
nof_list=np.arange(1,13)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

In [ ]:
df_lol.head()

In [ ]:
s_lol.head()

In [ ]:
fields_to_correlate = 'result champion1 champion2 champion3 champion4 champion5'.split()

In [ ]:
def correlation_in_league():
    df_lol.replace(' ', np.nan, inplace=True)
    corr = df_lol[fields_to_correlate].corr()
    print(corr.round(2))

correlation_in_league()

In [ ]:
df_lol['position1'].isnull().sum()

In [ ]:
#df_lol.dropna(inplace = True)
df_lol.isnull().sum()

In [ ]:
plt.figure(figsize=(12,10))
corr = df_lol[fields_to_correlate].corr()
sns.heatmap(corr, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
df_lol.head()

In [100]:
'''
Criando novo dataframe
'''
'''
lol_ML = pd.concat([ df_lol["result"], df_lol["side"], df_lol["team"],df_lol['gameid'],df_lol["ft"],df_lol['fbaron'],
         df_lol["champion1"], df_lol["position1"],df_lol["csdif1"],df_lol["xpdif1"],df_lol["golddif15_1"],
         df_lol["champion2"], df_lol["position2"],df_lol["csdif2"],df_lol["xpdif2"],df_lol["golddif15_2"], 
         df_lol["champion3"], df_lol["position3"],df_lol["csdif3"],df_lol["xpdif3"],df_lol["golddif15_3"],
         df_lol["champion4"], df_lol["position4"],df_lol["csdif4"],df_lol["xpdif4"],df_lol["golddif15_4"],
         df_lol["champion5"], df_lol["position5"],df_lol["csdif5"],df_lol["xpdif5"],df_lol["golddif15_5"],
         df_lol["enemy_champion1"], df_lol["enemy_champion2"], df_lol["enemy_champion3"],
         df_lol["enemy_champion4"], df_lol["enemy_champion5"]], axis = 1).copy()
'''

lol_ML = pd.concat([ df_lol["result"], df_lol["side"], df_lol["team"],df_lol['gameid'],df_lol["ft"],df_lol['fbaron'],
         df_lol["champion1"], df_lol["position1"],df_lol["csdif1"],df_lol["xpdif1"],df_lol["golddif15_1"],
         df_lol["champion2"], df_lol["position2"],df_lol["csdif2"],df_lol["xpdif2"],df_lol["golddif15_2"], 
         df_lol["champion3"], df_lol["position3"],df_lol["csdif3"],df_lol["xpdif3"],df_lol["golddif15_3"],
         df_lol["champion4"], df_lol["position4"],df_lol["csdif4"],df_lol["xpdif4"],df_lol["golddif15_4"],
         df_lol["champion5"], df_lol["position5"],df_lol["csdif5"],df_lol["xpdif5"],df_lol["golddif15_5"],
         df_lol["enemy_champion1"], df_lol["enemy_champion2"], df_lol["enemy_champion3"],
         df_lol["enemy_champion4"], df_lol["enemy_champion5"]], axis = 1).copy()



In [101]:
lol_ML.head()

,result,side,team,gameid,ft,fbaron,champion1,position1,csdif1,xpdif1,golddif15_1,champion2,position2,csdif2,xpdif2,golddif15_2,champion3,position3,csdif3,xpdif3,golddif15_3,champion4,position4,csdif4,xpdif4,golddif15_4,champion5,position5,csdif5,xpdif5,golddif15_5,enemy_champion1,enemy_champion2,enemy_champion3,enemy_champion4,enemy_champion5
200036,1,0,22,200036,1.0,1.0,41,5,24.0,503.0,993.0,24,1,-7.0,-505.0,547.0,58,2,8.0,758.0,1686.0,8,0,16.0,554.0,896.0,101,3,4.0,472.0,238.0,33,59,117,112,106
200039,0,0,38,200039,1.0,0.0,91,5,-2.0,37.0,-401.0,24,1,19.0,-21.0,371.0,100,2,-6.0,-144.0,-253.0,8,0,6.0,179.0,656.0,47,3,-5.0,-60.0,150.0,99,59,17,15,130
200048,0,0,9,200048,0.0,0.0,28,5,13.0,147.0,606.0,77,1,13.0,648.0,-18.0,58,2,-10.0,-409.0,145.0,44,0,-13.0,-256.0,-1475.0,47,3,-11.0,181.0,-204.0,33,54,78,15,106
200050,1,0,100,200050,1.0,1.0,30,5,-17.0,-250.0,-142.0,24,1,-5.0,-92.0,9.0,49,2,15.0,600.0,1195.0,15,0,-5.0,-618.0,1000.0,12,3,11.0,527.0,819.0,91,59,58,8,130
200051,0,0,16,200051,1.0,0.0,43,5,-8.0,-501.0,-798.0,119,1,13.0,-53.0,-336.0,49,2,-13.0,-340.0,-167.0,121,0,-12.0,-531.0,-137.0,102,3,2.0,-16.0,1564.0,53,59,118,15,106


In [102]:
lol_ML.dropna(inplace = True)
lol_ML.isnull().sum()

result             0
side               0
team               0
gameid             0
ft                 0
fbaron             0
champion1          0
position1          0
csdif1             0
xpdif1             0
golddif15_1        0
champion2          0
position2          0
csdif2             0
xpdif2             0
golddif15_2        0
champion3          0
position3          0
csdif3             0
xpdif3             0
golddif15_3        0
champion4          0
position4          0
csdif4             0
xpdif4             0
golddif15_4        0
champion5          0
position5          0
csdif5             0
xpdif5             0
golddif15_5        0
enemy_champion1    0
enemy_champion2    0
enemy_champion3    0
enemy_champion4    0
enemy_champion5    0
dtype: int64

In [103]:
lol_ML.shape

(2478, 36)

<h3> Treinando as variáveis de interesse com o modelo escolhido:</h3>
<p> Aqui, finalmente as variáveis de interesse serão de fato treinadas com o modelo preditivo que se saiu melhor na fase anterior e sua acurácia será calculada logo depois. Isso será feito para todas as variáveis de interesse ao mesmo tempo, com o objetivo de prever as chances de vitória.</p>

In [ ]:
#probabilites_variables = np.array(df_lol.iloc[:,4:])
#target = np.array(df_lol['result'])

In [104]:
probabilites_variables = np.array(lol_ML.iloc[:,4:])
target = np.array(lol_ML['result'])

In [105]:
'''
Aqui, de fato o dataset está sendo treinado de acordo com a divisão feita anteriormente feita 
anteriormente entre "features" e variáveis "alvo" (result). Existem vários modelos abaixo, mas somente
o modelo que se saiu melhor durante os testes do cross_validation_score ( melhor acurácia), será 
usado. 
''' 
resultados = []
n_folds = 20
k_folds = KFold(n_folds, shuffle=True, random_state = 1)
scores_teste = 0
scores_treino = 0



for linhas_treino, linhas_teste in k_folds.split(probabilites_variables,target):
    XTreino, XTeste = probabilites_variables[linhas_treino], probabilites_variables[linhas_teste]
    yTreino, yTeste = target[linhas_treino], target[linhas_teste]
    
    # Aqui o melhor modelo será escolhido para treinar as variáveis.    
    modelr = RandomForestClassifier(n_jobs = -1, n_estimators=500, max_depth = 6, min_samples_split = 2,
                                    min_samples_leaf = 2 , random_state = 1)
    #modelo = LogisticRegression(solver='liblinear', multi_class='ovr')
    #modeli = LinearDiscriminantAnalysis()
    #modelk = KNeighborsClassifier()
    #modeldtr = DecisionTreeRegressor(max_depth=8, min_samples_leaf=0.13, random_state=3)
    #modeld = DecisionTreeClassifier()
    #modelg = GaussianNB()
    #modelss = StandardScaler()
    #models = SVC(gamma='auto')
    
    
    modelr.fit(XTreino, yTreino)
    previsoes = modelr.predict(XTeste)
    resultados.append(accuracy_score(yTeste, previsoes))
    scores_treino += modelr.score(XTreino, yTreino)
    scores_teste += modelr.score(XTeste, yTeste)
    

    
#Calcula a acurácia    
    
print("Acurácias de cada pedaço dividido pelo dataset =>")
print(resultados)
print("\nMédia de todas as acurácias: ", np.mean(resultados))
print()
print("Média da pontuação de validação do modelo de treino: ", scores_treino/n_folds)
print("Média da pontuação de validação do modelo de teste: ", scores_teste/n_folds)



Acurácias de cada pedaço dividido pelo dataset =>
[0.8306451612903226, 0.8548387096774194, 0.8306451612903226, 0.8709677419354839, 0.8387096774193549, 0.7983870967741935, 0.8064516129032258, 0.8387096774193549, 0.8387096774193549, 0.8387096774193549, 0.8387096774193549, 0.7983870967741935, 0.8225806451612904, 0.8709677419354839, 0.8709677419354839, 0.9032258064516129, 0.8548387096774194, 0.8145161290322581, 0.8373983739837398, 0.8373983739837398]

Média de todas as acurácias:  0.8397882244951482

Média da pontuação de validação do modelo de treino:  0.8812284371183712
Média da pontuação de validação do modelo de teste:  0.839788224495148


<h1>Sem tunagem</h1>

In [ ]:
Random Forest (sem tunagem e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6005720964566931
    =>Com FT e sem FBaron: Média de todas as acurácias:  0.6518608513779529
    =>Com FT e com FBaron: Média de todas as acurácias:  0.7994197482297405


In [ ]:
Random Forest (sem tunagem e golddif 15 minutos):
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.656142347440945
    =>Com FT e sem FBaron: Média de todas as acurácias:  0.676519438976378
    =>Com FT e com FBaron: Média de todas as acurácias:  0.8087004982953057


<h1>TUNAGENS</h1>

<h3>Tunando n_estimators</h3>

In [ ]:
Random Forest (com tunagem - n_estimators = 100 -  e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6514456200787403

In [ ]:
=>MELHOR
Random Forest (com tunagem - n_estimators = 500 -  e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6658471020194072

In [ ]:
Random Forest (com tunagem - n_estimators = 1000 -  e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6622213480199318

<h3>Tunando max_depth</h3>

In [ ]:
Random Forest (com tunagem - n_estimators = 500, max_depth = 4 -  e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6602052189876738


In [ ]:
Random Forest (com tunagem - n_estimators = 500, max_depth = 5 -  e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6642309205350119


In [ ]:
=>MELHOR
Random Forest (com tunagem - n_estimators = 500, max_depth = 6 -  e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6646374246000524


In [ ]:
Random Forest (com tunagem - n_estimators = 500, max_depth = 7 -  e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6618115656963022


In [ ]:
Random Forest (com tunagem - n_estimators = 500, max_depth = 8 -  e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6573662470495671


In [ ]:
Random Forest (com tunagem - n_estimators = 500, max_depth = 10 -  e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6618148439548912


<h3>Tunando min_samples_split</h3>

In [ ]:
=>MELHOR
Random Forest (com tunagem - n_estimators = 500, max_depth = 6 , min_samples_split = 2 -  
               e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6646374246000524

In [ ]:
Random Forest (com tunagem - n_estimators = 500, max_depth = 6 , min_samples_split = 5 -  
               e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6610018358248099

In [ ]:
Random Forest (com tunagem - n_estimators = 500, max_depth = 6 , min_samples_split = 10 -  
               e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6630179648570679

In [ ]:
Random Forest (com tunagem - n_estimators = 500, max_depth = 6 , min_samples_split = 100 -  
               e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6589824285339628

<h3>Tunando min_samples_leaf</h3>

In [ ]:
Random Forest (com tunagem - n_estimators = 500, max_depth = 6 , min_samples_split = 2,
               min_samples_leaf = 1 - e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6646374246000524


In [ ]:
=>MELHOR
Random Forest (com tunagem - n_estimators = 500, max_depth = 6 , min_samples_split = 2,
               min_samples_leaf = 2 - e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.665043928665093

In [ ]:
Random Forest (com tunagem - n_estimators = 500, max_depth = 6 , min_samples_split = 2,
               min_samples_leaf = 5 - e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6626180173092053

In [ ]:
Random Forest (com tunagem - n_estimators = 500, max_depth = 6 , min_samples_split = 2,
               min_samples_leaf = 10 - e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6630081300813009

<h3>Realizando o teste com a diferença de gold aos 15 minutos</h3>

In [ ]:
Random Forest (com tunagem - n_estimators = 100, max_depth = 6 , min_samples_split = 2,
               min_samples_leaf = 2 - e golddif 15 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6964988198269079
    =>Com FT e sem FBaron: Média de todas as acurácias:  0.6973249409913453
    =>Com FT e com FBaron: Média de todas as acurácias:  0.8397882244951482


<h3>Abaixo está uma tentativa (falha) de usar o método Grid para procurar pela melhor tunagem.</h3>

In [ ]:

probabilites_variables = np.array(lol_ML.iloc[:,4:])
target = np.array(lol_ML['result'])

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(lol_ML, target, test_size=0.3, random_state=42)

In [ ]:
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7],
    'criterion' :['gini', 'entropy']
}

In [ ]:
from sklearn.model_selection import GridSearchCV

rfc=RandomForestClassifier(random_state=42)
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(x_train, y_train)


In [ ]:
CV_rfc.best_params_

In [ ]:
%pylab inline
pylab.hist(resultados)

In [ ]:
s_lol.head()

In [ ]:
'''
Esse gráfico representa a quantidade de vezes em que cada Campeão foi escolhido na partido.
'''

sortedPicks = w_lol['s_champion']


fig, (ax1) = plt.subplots(ncols=1, sharey=False, figsize=(15,30))
plt.xticks(rotation=90)
sns.countplot(y=sortedPicks, data=w_lol, ax=ax1)
ax1.set_title('Champion Picks')


In [ ]:
w_lol.head()

In [76]:
w_lol['result'].value_counts()

1    18702
0    18702
Name: result, dtype: int64

In [77]:
'''
Lista dos 10 campeões mais escolhidos na partida
'''
w_lol['s_champion'].value_counts().head(11)


           6234
gragas     1110
varus      1044
ashe        994
orianna     877
karma       852
leesin      833
khazix      803
thresh      797
maokai      771
elise       767
Name: s_champion, dtype: int64

In [78]:
'''
Conta o número de vitórias dos 10 campeões mais escolhidos
'''
count_gragas = 0
count_varus = 0
count_ashe = 0
count_orianna = 0
count_karma = 0
count_leesin = 0
count_khazix = 0
count_thresh = 0
count_maokai = 0
count_elise = 0

for index, row in w_lol.iterrows():
    if(row.s_champion == 'gragas' and row.result == 1):
        count_gragas += 1
        
    elif(row.s_champion == 'varus' and row.result == 1):
        count_varus += 1
        
    elif(row.s_champion == 'ashe' and row.result == 1):
        count_ashe += 1
        
    elif(row.s_champion == 'orianna' and row.result == 1):
        count_orianna += 1
        
    elif(row.s_champion == 'karma' and row.result == 1):
        count_karma += 1
        
    elif(row.s_champion == 'leesin' and row.result == 1):
        count_leesin += 1
        
    elif(row.s_champion == 'khazix' and row.result == 1):
        count_khazix += 1
        
    elif(row.s_champion == 'thresh' and row.result == 1):
        count_thresh += 1
        
    elif(row.s_champion == 'maokai' and row.result == 1):
        count_maokai += 1
        
    elif(row.s_champion == 'elise' and row.result == 1):
        count_elise += 1
        
        
        


In [79]:
'''
Conta o número de derrotas dos 10 campeões mais escolhidos
'''
countd_gragas = 0
countd_varus = 0
countd_ashe = 0
countd_orianna = 0
countd_karma = 0
countd_leesin = 0
countd_khazix = 0
countd_thresh = 0
countd_maokai = 0
countd_elise = 0

for index, row in w_lol.iterrows():
    if(row.s_champion == 'gragas' and row.result == 0):
        countd_gragas += 1
        
    elif(row.s_champion == 'varus' and row.result == 0):
        countd_varus += 1
        
    elif(row.s_champion == 'ashe' and row.result == 0):
        countd_ashe += 1
        
    elif(row.s_champion == 'orianna' and row.result == 0):
        countd_orianna += 1
        
    elif(row.s_champion == 'karma' and row.result == 0):
        countd_karma += 1
        
    elif(row.s_champion == 'leesin' and row.result == 0):
        countd_leesin += 1
        
    elif(row.s_champion == 'khazix' and row.result == 0):
        countd_khazix += 1
        
    elif(row.s_champion == 'thresh' and row.result == 0):
        countd_thresh += 1
        
    elif(row.s_champion == 'maokai' and row.result == 0):
        countd_maokai += 1
        
    elif(row.s_champion == 'elise' and row.result == 0):
        countd_elise += 1
        
        
        


In [80]:
'''
Número de derrotas e vitórias dos campeões mais escolhidos

'''

print("Gragas  => Derrotas: {}, Vitórias: {}".format(count_gragas, countd_gragas))
print("Varus   => Derrotas: {}, Vitórias: {}".format(count_varus, countd_varus))
print("Ashe    => Derrotas: {}, Vitórias: {}".format(count_ashe, countd_ashe))
print("Orianna => Derrotas: {}, Vitórias: {}".format(count_orianna, countd_orianna))
print("Karma   => Derrotas: {}, Vitórias: {}".format(count_karma, countd_karma))
print("Leesin  => Derrotas: {}, Vitórias: {}".format(count_leesin, countd_leesin))
print("Khaziz  => Derrotas: {}, Vitórias: {}".format(count_khazix, countd_khazix))
print("Thresh  => Derrotas: {}, Vitórias: {}".format(count_thresh, countd_thresh))
print("Maokai  => Derrotas: {}, Vitórias: {}".format(count_maokai, countd_maokai))
print("Elise   => Derrotas: {}, Vitórias: {}".format(count_elise, countd_elise))

Gragas  => Derrotas: 564, Vitórias: 546
Varus   => Derrotas: 496, Vitórias: 548
Ashe    => Derrotas: 492, Vitórias: 502
Orianna => Derrotas: 447, Vitórias: 430
Karma   => Derrotas: 389, Vitórias: 463
Leesin  => Derrotas: 401, Vitórias: 432
Khaziz  => Derrotas: 413, Vitórias: 390
Thresh  => Derrotas: 415, Vitórias: 382
Maokai  => Derrotas: 393, Vitórias: 378
Elise   => Derrotas: 403, Vitórias: 364
